In [ ]:
import pandas as pd
import numpy as np
import requests
import time
from IPython.display import Image
from IPython.display import display, HTML
from IPython.display import clear_output

In [ ]:
def value_isolator(response):
    return [x['value'] for x in response.json()['cards']]

def value_converter(cards):
    if cards.lower() in ['jack', 'queen', 'king', 'j', 'q', 'k']:
        return 10
    elif cards=='10':
        return 10
    elif cards.lower() in['ace','a']:
        return 11
    else:
        return int(cards)

def player_game(status, values):
    count=sum([value_converter(card) for card in values])
    ace=0
    if values==['ACE','ACE'] or values==['A','A']:
        ace=2
    elif 'ACE' in values or 'A' in values:
        ace=1

    while status==0:
        #Aces can be 11 or 1 in blackjack
        if ace==1 and count>21:
          count-=10
          ace=0
        elif ace==2 and count>21:
          count-=10
          ace=1
        #Check if we are bust
        elif count>21:
          status=2
        print(f'\nYour current count is {count}')
        time.sleep(1)

        #If we are not bust, we have the option to hit or stand
        if status!=2:
          action=input('\nHit or Stand? ')
          if action.lower()in['stand','s']:
            status=1
          elif action.lower()in['hit','h']:
            #Pull new card
            cards=requests.get(f'https://deckofcardsapi.com/api/deck/{deck_id}/draw/?count=1')

            #Display the new card
            image_url = cards.json()['cards'][0]['image']
            display(Image(url=image_url))

            #Get the number from the card and update the count
            values=value_isolator(cards)
            count+=sum([value_converter(card) for card in values])

            #Will need to recheck if we got an ace
            if 'ACE' in values:
                ace+=1
        else:
          print("BUST")

    return [status, count]

def dealer_game(values):
    count=sum([value_converter(card) for card in values])
    ace=0
    status=0
    if values==['ACE','ACE'] or values==['A','A']:
        ace=2
    elif 'ACE' in values or 'A' in values:
        ace=1

    while status==0:

        #Aces can be 11 or 1 in blackjack
        if ace==1 and count>21:
          count-=10
          ace=0
        elif ace==2 and count>21:
          count-=10
          ace=1

        #Dealer always stands on 17
        if count>16:
          status=1
        if count>21:
          status=2
        if count==17 and ace!=0:
          status=0

        print(f'Dealer\'s count is {count}')
        time.sleep(1)

        #If dealer's count<17, they will hit
        if status==0:
            #Pull new card
            dealer_card=requests.get(f'https://deckofcardsapi.com/api/deck/{deck_id}/draw/?count=1')

            #Display the new card
            image_url = dealer_card.json()['cards'][0]['image']
            display(Image(url=image_url))

            #Get the number from the card and update the count
            values=value_isolator(dealer_card)
            count+=sum([value_converter(card) for card in values])

            #Will need to recheck if we got an ace
            if 'ACE' in values:
                ace=1
        elif status==2:
          print("BUST")
        elif status==1:
          print(f'Dealer got {count}')

    return [status, count]

In [ ]:
def blackjack():
    #Pull a deck
    deck=response.json()
    deck_id=deck['deck_id']

    #Get two cards for player
    cards=requests.get(f'https://deckofcardsapi.com/api/deck/{deck_id}/draw/?count=2')


    #Display the images for these two cards
    url_list=[cards.json()['cards'][i]['image'] for i in range(len(cards.json()['cards']))]
    html_code = f'<img src="{url_list[0]}" style="float: left; margin-right: 10px;"><img src="{url_list[1]}" style="float: left;">'
    display(HTML(html_code))
    print('\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n Dealer\'s card:')

    #Get one card for dealer and display
    dealer_card=requests.get(f'https://deckofcardsapi.com/api/deck/{deck_id}/draw/?count=1')
    image_url = dealer_card.json()['cards'][0]['image']
    display(Image(url=image_url))

    #Convert responses to workable format:

    cards=value_isolator(cards)
    dealer_card=value_isolator(dealer_card)

    split='No'

    #Check for split
    if len(set(cards))==1:
      split=input('Would You Like to Split? (Y/N) ')

    if split in ['Yes', 'Y' ,'yes', 'y']:
        print('OK, first hand')
        hand1=player_game(0, [cards[0]])
        print('Now for the other card')
        hand2=player_game(0, [cards[1]])
        player_result1= hand1[0]
        player_count1=hand1[1]
        player_result2= hand2[0]
        player_count2=hand2[1]

        if player_result1==2 and player_result2==2:
          return ['LOSS x2', 'TIE x0']
        else:
          dealer_hand=dealer_game(dealer_card)
          dealer_result=dealer_hand[0]
          dealer_count=dealer_hand[1]

          if dealer_result==2:
            wins= sum([result==1 for result in [player_result1, player_result2]])
            print(f'You won {wins} of the 2 hands')
            return [f'WIN x{wins}', 'TIE x0']
          elif dealer_result==1:
            ties=sum([result==dealer_count for result in [player_count1, player_count2]])
            wins=sum([dealer_count<result<22 for result in [player_count1, player_count2]])
            print(f'You got {player_count1} and {player_count2}, dealer got {dealer_count}.')
            print(f'You won {wins} and tied {ties} of the 2 hands')
            return [f'WIN x{wins}', f'TIE x{ties}']
    else:
        #Normal Game
        hand = player_game(0, cards)
        player_result= hand[0]
        player_count=hand[1]

        if player_result==2:
          return ['LOSS']
        else:
          dealer_hand=dealer_game(dealer_card)
          dealer_result=dealer_hand[0]
          dealer_count=dealer_hand[1]

          if player_result==1 and dealer_result==2:
            print("You Win")
            return ["WIN"]
          if player_result==1 and dealer_result==1 and player_count>dealer_count:
            print("You Win")
            return ["WIN"]
          if player_result==1 and dealer_result==1 and player_count==dealer_count:
            print("Tie, bet returned")
            return ["TIE"]
          else:
            print("You Lose")
            return ["LOSS"]





Easy way to play hands on repeat, if you would like to keep track of you previous hand, just delete the clear_output line

In [ ]:
response=requests.get('https://deckofcardsapi.com/api/deck/new/shuffle/?deck_count=6')
deck=response.json()
deck_id=deck['deck_id']
hand_number=1
cash=0
#bet=int(input('Place you bet: '))
bet=10
try:
  while True:
    if hand_number % 25 == 0:
      response=requests.get(f'https://deckofcardsapi.com/api/deck/{deck_id}/shuffle/')
      print("NEW DECK")
      time.sleep(5)
    hand=blackjack()
    if len(hand)==1:
      if hand==['LOSS']:
        cash-=bet
      elif hand==['WIN']:
        cash+=bet
    if len(hand)==2:
      if hand==['LOSS x2', 'TIE x0'] or hand == ['WIN x0', 'TIE x0']:
        cash-=2*bet
      elif hand==['WIN x2', 'TIE x0']:
        cash+=2*bet
      elif hand ==['WIN x1', 'TIE x1']:
        cash+=bet
      elif hand == ['WIN x0', 'TIE x1']:
        cash-=bet
    time.sleep(2.5)
    clear_output(wait=True)
    hand_number+=1
    print(f'Hand Number:{hand_number}, Cash Balance: {cash}')
except KeyboardInterrupt:
    print('Ended')